- 保监会 分类模型 2 训练 

# 基本设置

In [1]:
##load packages, needed
# encoding=utf-8

import jieba
from jieba import analyse
import sys
import re
import time
import string
from sklearn import feature_extraction
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif 
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

import pickle

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor
import dict_dbutils
import os
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from pandas.io import sql
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
class StatsFeatures_cor(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("corpus/neg_words.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content)
        f.close()
        dict_org=dict_dbutils.get_dicts()
        self.org=set(dict_org.keys())
        
        

    def fit(self, X, y=None):
        return self

    def getcnt(self,x):        
        return len(list(set(x)))

    def getnegcnt(self,x):
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt
    def getorgcnttf(self,x):
        orgcnt=0
        orgtf=0
        words = x.split()
        words_set=set(words)
        for w in words_set:
            if w in self.org:
                orgcnt = orgcnt+1
                orgtf=orgtf+words.count(w)
        if(orgcnt>0):
            return orgcnt,orgtf
        else:
            return orgcnt,orgtf
    
    
    def transform(self, X):
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
            orgcnt,orgtf=self.getorgcnttf(x)
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length,
                         self.getnegcnt(x),self.getnegcnt(x)/length,
                         orgcnt,orgtf])            
        return data

In [3]:
def getkeywords(X, N = 1000):
    '''
    训练时生成，合并所有记录，取N个关键词
    '''
    textrank = analyse.textrank

    text_combined = ' '.join(X)
    keywords = textrank(text_combined, topK = N)
    print('keywords num: ', len(keywords))
    if len(keywords) < N : 
        N  = len(keywords)

    if keywords:
        f = open("corpus/keywords.txt","w+", encoding='UTF-8')
        for content in keywords:
            content = content.strip()
            f.write(content + '\n')
        f.close()

In [4]:
class Statskeywords(BaseEstimator, TransformerMixin):
    
    def __init__(self, topk = 100):
        self.topk = topk
#         print(self.topk)
        self.keywords = set()
        f = open("corpus/keywords.txt","r+", encoding='UTF-8')
        num = 0
        for content in f:
            if num < topk:
                self.keywords.add(content.strip().replace('\n', ''))
            num += 1
        f.close() 
        
        #初始化字典liwc
        self.liwc = {} 
        f2 = open("corpus/scliwc.txt",'r', encoding = 'gb18030')
        for ii in f2:     #ii在scliwc.txt中循环
            i = ii.strip().split() 
            self.liwc[i[0]] = i[1:len(i)]
        f2.close      
        
        self.category = set()
        for i in list(self.liwc.values()):
            for j in i:
                self.category.add(j)        
    
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X):
        '''
        文本中关键词的词频
        '''                        
        data = []
        for x in X:
            words = x.split()
            word_tf = []
            keycnt = 0
            for kw in self.keywords:
                word_tf.append(words.count(kw)) # 各个关键词的词频
                if kw in words:keycnt+=1
            word_tf.append(keycnt) # 关键词的个数
            
            psy = []
            for w in words:
                if w in self.liwc: #是否liwc字典包含分词结果列表words的哪些分词
                    psy += self.liwc[w]  
            cat_tf = []
            for cat in self.category:
                cat_tf.append(psy.count(cat))                
                
            data.append(word_tf + cat_tf)            
        return data        

## 上一版模型读入

In [5]:
# 上一版模型
from sklearn.externals import joblib
pipeline_old = joblib.load( "model/circ_8classifier_0730_1.pkl.z") 

# 导入数据

In [22]:
title = []
filename = 'data/titles.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    title.append(f.strip().replace('\n', ''))
fid.close()
print(len(title))

10530


In [23]:
title[:2]

['做好 规划 不当 月光族 守财奴', '车险 代理 前景 如何 信合 财富 事业 迈上 高度']

In [24]:
content = []
filename = 'data/contents.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    content.append(f.strip().replace('\n', ''))
fid.close()
print(len(content))
# content[:2]

10530


In [25]:
title_content = [t + ' ' + c for t,c in zip(title, content)]
print(len(title_content))

10530


In [26]:
label_dic={'监管':1,'行业':2,'产品销售':3,'资本市场':4,'公司内部管理':5,'消费服务':6,'其他相关报道':7,'噪音':8}
class_name_dict = {v: k for k, v in label_dic.items()}
class_name_dict

{1: '监管',
 2: '行业',
 3: '产品销售',
 4: '资本市场',
 5: '公司内部管理',
 6: '消费服务',
 7: '其他相关报道',
 8: '噪音'}

In [27]:
label = []
filename = 'data/labels.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    label.append(f.strip().replace('\n', ''))
fid.close()
print(len(label))
label[:5]

10530


['噪音', '噪音', '产品销售', '噪音', '消费服务']

In [28]:
label = [label_dic[l] for l in label]
label[:5]

[8, 8, 3, 8, 6]

# 训练模型

## 分割训练集、测试集

In [29]:
X_train, X_test, y_train, y_test = train_test_split(title_content, label, test_size=0.3, random_state=42)
print('训练集：',len(y_train))
print('训练集-各类数量：',Counter(y_train))
print('测试集：',len(y_test))
print('测试集-各类数量：',Counter(y_test))

训练集： 7371
训练集-各类数量： Counter({8: 2358, 6: 1117, 1: 1056, 7: 1005, 5: 638, 2: 520, 4: 410, 3: 267})
测试集： 3159
测试集-各类数量： Counter({8: 1018, 6: 456, 7: 451, 1: 432, 5: 263, 2: 243, 4: 173, 3: 123})


## 生成词典

In [14]:
if 0: 
    from jieba import analyse
    getkeywords(title_content, N = 1000)

### 分类生成词典

In [30]:
# j = 1
N = 200
class_key_dict = {}
key_dict = {}
for j in range(1,9):
    str_list = [m for m,n in zip(title_content, label) if n == j]
    print(len(str_list))
    print(label.count(j))
    textrank = analyse.textrank
    text_combined = ' '.join(str_list)
    keywords = textrank(text_combined, topK = N)
    print('keywords num: ', len(keywords))
#     print('%s -- %s: '%(j, class_name_dict[j]) , keywords)
    class_key_dict[j] = keywords
    for key in keywords:
        if key not in key_dict:            
            key_dict[key] = 1
        else :
            key_dict[key] += 1
    

1488
1488
keywords num:  200
763
763
keywords num:  200
390
390
keywords num:  200
583
583
keywords num:  200
901
901
keywords num:  200
1573
1573
keywords num:  200
1456
1456
keywords num:  200
3376
3376
keywords num:  200


In [31]:
key_count_list = [k for k,v in key_dict.items() if v > 4]
print(len(key_count_list))
print(len(key_dict.values()))
print(list(key_dict.values()))

101
697
[7, 7, 8, 8, 7, 8, 8, 7, 7, 7, 8, 8, 3, 7, 5, 8, 1, 8, 2, 8, 7, 8, 4, 8, 3, 8, 8, 8, 8, 2, 5, 8, 2, 8, 2, 3, 8, 3, 7, 1, 4, 7, 8, 8, 1, 6, 1, 2, 8, 3, 6, 1, 8, 5, 7, 2, 1, 4, 7, 1, 7, 4, 1, 4, 4, 3, 1, 2, 8, 3, 8, 6, 6, 4, 1, 7, 1, 2, 2, 7, 3, 1, 5, 2, 1, 4, 6, 1, 2, 1, 4, 3, 2, 3, 8, 7, 4, 7, 6, 1, 5, 4, 6, 1, 5, 1, 1, 6, 1, 2, 4, 1, 7, 6, 7, 1, 2, 6, 3, 3, 1, 4, 5, 1, 6, 7, 5, 1, 1, 2, 1, 2, 1, 5, 7, 2, 2, 7, 6, 1, 2, 5, 1, 2, 2, 7, 7, 1, 2, 1, 1, 2, 1, 1, 7, 1, 3, 3, 6, 2, 5, 6, 1, 1, 4, 2, 7, 3, 1, 5, 1, 1, 5, 4, 1, 1, 5, 4, 1, 4, 1, 2, 7, 8, 4, 1, 5, 1, 4, 4, 3, 1, 1, 1, 5, 3, 2, 1, 2, 2, 5, 4, 4, 5, 2, 4, 6, 2, 2, 3, 3, 5, 3, 2, 6, 2, 2, 4, 3, 5, 3, 2, 3, 5, 3, 5, 4, 5, 5, 2, 1, 2, 6, 7, 6, 3, 5, 1, 3, 1, 3, 2, 4, 2, 5, 1, 5, 3, 1, 1, 2, 1, 3, 1, 3, 3, 1, 2, 1, 6, 2, 1, 4, 2, 3, 1, 3, 2, 4, 2, 1, 1, 3, 3, 1, 1, 2, 4, 2, 4, 3, 2, 2, 3, 1, 3, 3, 5, 2, 1, 2, 1, 2, 3, 1, 2, 3, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 3, 1, 1, 1, 1, 1, 2, 4, 1, 3, 4, 4, 2, 1, 2, 1, 1, 1, 2, 1, 1, 3, 1, 2

In [32]:
keywords = []
for j in range(1,9):
    print('%s --'%j)
    keyword_list = class_key_dict[j]
    print('去除前：', len(keyword_list))
    for k in key_count_list:        
        if k in keyword_list:
            keyword_list.remove(k)
    print('去除后：', len(keyword_list)) 
    
    keywords += keyword_list

print('len(keywords): ', len(keywords))
f = open("corpus/keywords.txt","w+", encoding='UTF-8')
for content in keywords:
    content = content.strip()
    f.write(content + '\n')
f.close()    

1 --
去除前： 200
去除后： 117
2 --
去除前： 200
去除后： 99
3 --
去除前： 200
去除后： 111
4 --
去除前： 200
去除后： 123
5 --
去除前： 200
去除后： 111
6 --
去除前： 200
去除后： 139
7 --
去除前： 200
去除后： 123
8 --
去除前： 200
去除后： 119
len(keywords):  942


## LDA: 主题模型选特征

In [81]:
from sklearn.decomposition import LatentDirichletAllocation

tf_vectorizer  = CountVectorizer(max_df=0.95, min_df=2)
tf = tf_vectorizer .fit_transform(X_train)

In [82]:
len(tf.data), tf.data

(1613520, array([1, 1, 1, ..., 1, 6, 1], dtype=int64))

In [83]:
tf_feature_names = tf_vectorizer.get_feature_names()
print(tf_feature_names[:10])
len(tf_feature_names)

['aa', 'aaa', 'aaaa', 'aaaaa', 'aaf', 'aa制', 'ab', 'abb', 'abbott', 'abbvie']


60262

In [84]:
lda = LatentDirichletAllocation(n_topics=20,
                                max_iter=50,
                                learning_method='batch',
                                random_state=0)
docres = lda.fit_transform(tf)

In [85]:
docres

array([[3.90625003e-04, 9.92578125e-01, 3.90625003e-04, ...,
        3.90625095e-04, 3.90625002e-04, 3.90625004e-04],
       [4.50450455e-04, 1.28539163e-01, 7.50649189e-01, ...,
        4.50450465e-04, 4.50450457e-04, 4.50450459e-04],
       [6.44618417e-02, 7.77604998e-05, 7.77604987e-05, ...,
        2.78697680e-02, 2.06295485e-02, 7.77604990e-05],
       ...,
       [1.21359227e-04, 1.21359227e-04, 1.21359225e-04, ...,
        1.21359226e-04, 1.21359226e-04, 1.21359225e-04],
       [4.32523919e-02, 5.10204087e-04, 5.10204084e-04, ...,
        3.71540782e-01, 5.10204085e-04, 5.10204082e-04],
       [9.32835840e-05, 9.32835834e-05, 9.32835835e-05, ...,
        9.32835830e-05, 1.79299844e-01, 2.00158643e-01]])

In [86]:
lda.components_

array([[5.00000001e-02, 5.56502649e+00, 5.00000000e-02, ...,
        5.00000000e-02, 5.00000000e-02, 5.00000006e-02],
       [1.01622379e+01, 5.00000005e-02, 5.00000000e-02, ...,
        5.00000006e-02, 5.00000003e-02, 5.00000002e-02],
       [5.00000003e-02, 5.00000002e-02, 5.00000000e-02, ...,
        5.00000000e-02, 5.00000000e-02, 5.00000040e-02],
       ...,
       [5.00000001e-02, 5.00000004e-02, 5.00000000e-02, ...,
        5.00000000e-02, 5.00000000e-02, 5.00000001e-02],
       [5.00000012e-02, 5.00000010e-02, 5.00000000e-02, ...,
        5.00000028e-02, 5.00000007e-02, 5.00000001e-02],
       [2.06829458e+02, 5.76482636e+01, 5.00000000e-02, ...,
        5.00000000e-02, 5.00000000e-02, 5.00000002e-02]])

In [87]:
def print_top_words(model, feature_names, n_top_words):
    #打印每个主题下权重较高的term
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    #打印主题-词语分布矩阵
    print(model.components_)

n_top_words=20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
招标 项目 文件 投标 采购 图片 工程 提供 公告 投标人 要求 时间 单位 资格 企业 具有 注册 条件 原件 报名
Topic #1:
公司 亿元 保险 增长 业务 保险公司 同比 股东 产品 寿险 保费 保费收入 集团 车险 发展 市场 中国 显示 其中 行业
Topic #2:
社保 部门 养老保险 缴费 征收 税务 企业 就业 参保 统一 职工 基本 养老金 医疗保险 社保费 缴纳 养老 规定 个人 医疗
Topic #3:
招聘 人员 报名 岗位 面试 资格 工作 笔试 时间 体检 聘用 条件 进行 考试 公开 教师 成绩 应聘 专业 具有
Topic #4:
疫苗 公告 中国 企业 国家 公司 生物 长生 汽车 美国 发布 项目 事件 一路 一带 新闻 亿美元 国际 上半年 美元
Topic #5:
教师 讲学 工作 答案 点击 解析 查看 招募 学校 计划 服务 电梯 教育 农村 根据 暂无 救援 活动 退休 单位
Topic #6:
中国 公司 美国 集团 杭州 日本 世界 泰康 有限公司 香港 德国 英国 光伏 服饰 group 国家 融资担保 医养 国际 法国
Topic #7:
工作 企业 建设 服务 开展 标准 发展 推进 安全 加强 金融 问题 生产 要求 部门 管理 制度 改革 实施 机制
Topic #8:
投资 股权 保险 计划 ff 保险资金 企业 关联交易 设立 业务 通知 实体 资产 融资 经济 保监会 重大 方式 管理 项目
Topic #9:
活动 孩子 一个 学生 生活 教育 扶贫 保险 社会 图片 没有 家庭 学校 宣传 富德生命人寿 老师 家长 开始 作品 员工
Topic #10:
分公司 云南 社会 云南省 寿险 中国人寿保险股份有限公司 管理 招聘 任职 资格 销售 关于 支公司 公司 批复 昆明市 核准 之日起 许可 中心
Topic #11:
工作 招聘 要求 经验 以上 电话 联系电话 公司 工资 优先 待遇 地址 相关 面议 岗位 销售 能力 职位 联系 薪资
Topic #12:
保险 保险公司 理赔 如果 保障 客户 没有 一个 产品 投保 购买 需要 图片 很多 因为 保费 什么 保单 医院 可能
Topic #13:
活动 车辆 责任 承担 旅游 费用 赔偿 声明 平台 事

In [88]:
doc_topic_dist = lda.transform(tf)

In [89]:
lda.perplexity(tf)

2737.4835277309144

In [ ]:
from time import time
n_topics = [200,] #range(20, 75, 5)
perplexityLst = [1.0]*len(n_topics)

#训练LDA并打印训练时间
lda_models = []
for idx, n_topic in enumerate(n_topics):
    lda = LatentDirichletAllocation(n_topics=n_topic,
                                    max_iter=1000,
                                    learning_method='batch',
                                    evaluate_every=200,
#                                    perp_tol=0.1, #default                                       
#                                    doc_topic_prior=1/n_topic, #default
#                                    topic_word_prior=1/n_topic, #default
                                    verbose=0)
    t0 = time()
    lda.fit(tf)
    perplexityLst[idx] = lda.perplexity(tf)
    lda_models.append(lda)
    print("# of Topic: %d, " % n_topics[idx])
    print("done in %0.3fs, N_iter %d, " % ((time() - t0), lda.n_iter_))
    print("Perplexity Score %0.3f" % perplexityLst[idx])

In [ ]:
#打印最佳模型
best_index = perplexityLst.index(min(perplexityLst))
best_n_topic = n_topics[best_index]
best_model = lda_models[best_index]
print("Best # of Topic: ", best_n_topic)

## xgboost

### train

In [33]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=8))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9968796635463302


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...tate=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

### test

In [34]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.8448876226654004
             precision    recall  f1-score   support

          1       0.79      0.87      0.83       432
          2       0.69      0.61      0.65       243
          3       0.77      0.49      0.60       123
          4       0.82      0.75      0.78       173
          5       0.81      0.79      0.80       263
          6       0.92      0.92      0.92       456
          7       0.87      0.88      0.88       451
          8       0.87      0.92      0.89      1018

avg / total       0.84      0.84      0.84      3159

confusion_matrix: 
[[375  14   3   3  10   1   1  25]
 [ 26 148   4   5   7  12  17  24]
 [  5   5  60   0   7   9  13  24]
 [  9  15   0 130   3   0   0  16]
 [ 15  10   3   1 207   5   9  13]
 [  0   5   1   0   3 419  11  17]
 [  7  10   4   2   8   4 398  18]
 [ 38   6   3  18  10   4   7 932]]


In [32]:
# 上一版模型 
y_pred_class = pipeline_old.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.9001331557922769
             precision    recall  f1-score   support

          1       0.90      0.95      0.93       279
          2       0.76      0.69      0.73        65
          3       0.94      0.51      0.66        63
          4       0.95      0.82      0.88        89
          5       0.88      0.78      0.83       128
          6       0.96      0.81      0.88        59
          7       0.89      0.91      0.90       143
          8       0.91      0.97      0.94       676

avg / total       0.90      0.90      0.90      1502

confusion_matrix: 
[[266   4   0   0   3   1   0   5]
 [  4  45   0   1   1   0   4  10]
 [  1   2  32   0   1   1   5  21]
 [  2   1   0  73   0   0   0  13]
 [ 10   0   2   0 100   0   5  11]
 [  0   2   0   0   2  48   1   6]
 [  6   2   0   0   2   0 130   3]
 [  6   3   0   3   5   0   1 658]]


### KFold

In [14]:
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=0)
title_content = np.array(title_content)
label = np.array(label)

In [15]:
num = 0
for train_index , test_index in kf.split(title_content):
    print('---- %s'%(num+1))
    X_train,X_test = title_content[train_index], title_content[test_index]
    y_train,y_test = label[train_index], label[test_index]
    
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('tf_idf', Pipeline([
                ('counts', CountVectorizer(max_df=0.95, min_df=2)),
                ('tf_idf', TfidfTransformer()),
                ('chi', SelectKBest(chi2, k=20000))
            ])),
            ('tf', Statskeywords(topk = 1000)),
            ('len_stats', StatsFeatures_cor())
        ])),
        ('standard', StandardScaler(with_mean=False)),
        ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=8))
    ])
    pipeline.fit(X_train, y_train)
    print(pipeline.score(X_train, y_train))    
    
    y_pred_class = pipeline.predict(X_test)
    print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
    print(metrics.classification_report(y_test, y_pred_class))
    print('confusion_matrix: ')
    print( metrics.confusion_matrix(y_test, y_pred_class))    
    
    num += 1

---- 1
0.9991673605328892
accuracy_score:  0.8408788282290279
             precision    recall  f1-score   support

          1       0.83      0.88      0.86       286
          2       0.52      0.41      0.46        58
          3       0.76      0.50      0.60        58
          4       0.88      0.78      0.83        98
          5       0.82      0.75      0.78       126
          6       0.71      0.80      0.75        59
          7       0.85      0.81      0.83       151
          8       0.88      0.93      0.90       666

avg / total       0.84      0.84      0.84      1502

confusion_matrix: 
[[253   5   2   1   1   0   2  22]
 [ 11  24   0   0   2   3   1  17]
 [  6   2  29   1   4   2   5   9]
 [  2   3   0  76   6   0   1  10]
 [  7   3   1   1  94   3   8   9]
 [  0   1   1   0   1  47   1   8]
 [  3   3   1   0   3   8 122  11]
 [ 21   5   4   7   4   3   4 618]]
---- 2
0.9993338884263114
accuracy_score:  0.8468708388814914
             precision    recall  f1-score 

# 调参

In [20]:
score_1_best_para = {}
score_2_best_para = {}
cv = 3
score_1 = 'accuracy'
score_2 = 'recall_macro'

In [21]:
combined_features = FeatureUnion([
                                    ('tf_idf', Pipeline([
                                        ('counts', CountVectorizer()),
                                        ('tfidf', TfidfTransformer()),
                                        ('chi', SelectKBest(chi2, k=20000))
                                        ])),
                                    ('tf', Statskeywords(topk = 100)),
                                    ('len_stats', StatsFeatures_cor())
                                ])

## topk、chi_k

In [22]:
pipeline = Pipeline([('features', combined_features), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0,
                                                  objective='multi:softmax', num_class=8))])

param_grid_0 = dict(features__tf_idf__chi__k = [2000, 5000, 20000, 40000 ],
                   features__tf__topk=[50,100, 500]) 
param_grid_0

{'features__tf_idf__chi__k': [2000, 5000, 20000, 40000],
 'features__tf__topk': [50, 100, 500]}

In [23]:
clf = GridSearchCV(pipeline, param_grid=param_grid_0, 
                   cv = cv, scoring=score_1, n_jobs=-1, verbose=10)
clf.fit(X_train, y_train)
score_1_best_para['chi__k'] = clf.best_params_['features__tf_idf__chi__k']
score_1_best_para['topk'] = clf.best_params_['features__tf__topk']
print('score_1_chi__k: ', score_1_best_para['chi__k'])
print('score_1_topk: ', score_1_best_para['topk'])
clf.grid_scores_, clf.best_params_, clf.best_score_

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] features__tf__topk=50, features__tf_idf__chi__k=2000 ............
[CV] features__tf__topk=50, features__tf_idf__chi__k=2000 ............
[CV] features__tf__topk=50, features__tf_idf__chi__k=2000 ............
[CV] features__tf__topk=50, features__tf_idf__chi__k=5000 ............
[CV]  features__tf__topk=50, features__tf_idf__chi__k=2000, score=0.8128921848260126, total= 2.9min
[CV] features__tf__topk=50, features__tf_idf__chi__k=5000 ............
[CV]  features__tf__topk=50, features__tf_idf__chi__k=2000, score=0.8187001140250855, total= 3.0min
[CV] features__tf__topk=50, features__tf_idf__chi__k=5000 ............
[CV]  features__tf__topk=50, features__tf_idf__chi__k=2000, score=0.8148571428571428, total= 3.1min
[CV] features__tf__topk=50, features__tf_idf__chi__k=20000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=5000, score=0.822690992018244, total= 4.0min
[CV] features__tf__topk=50, features__tf_id

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  9.4min


[CV]  features__tf__topk=50, features__tf_idf__chi__k=5000, score=0.8148571428571428, total= 5.1min
[CV] features__tf__topk=50, features__tf_idf__chi__k=40000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=20000, score=0.822690992018244, total= 8.4min
[CV] features__tf__topk=50, features__tf_idf__chi__k=40000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=20000, score=0.814033086138049, total= 8.1min
[CV] features__tf__topk=50, features__tf_idf__chi__k=40000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=20000, score=0.8154285714285714, total= 8.7min
[CV] features__tf__topk=100, features__tf_idf__chi__k=2000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=40000, score=0.822690992018244, total=10.4min


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 20.9min


[CV] features__tf__topk=100, features__tf_idf__chi__k=2000 ...........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=2000, score=0.8187001140250855, total= 3.2min
[CV] features__tf__topk=100, features__tf_idf__chi__k=2000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=40000, score=0.8134626354820308, total= 9.3min
[CV] features__tf__topk=100, features__tf_idf__chi__k=5000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=40000, score=0.8114285714285714, total= 9.1min
[CV] features__tf__topk=100, features__tf_idf__chi__k=5000 ...........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=2000, score=0.8128921848260126, total= 3.2min
[CV] features__tf__topk=100, features__tf_idf__chi__k=5000 ...........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=2000, score=0.8148571428571428, total= 3.4min
[CV] features__tf__topk=100, features__tf_idf__chi__k=20000 ..........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=5000, score=0.

...

[Parallel(n_jobs=-1)]: Done  33 out of  36 | elapsed: 58.6min remaining:  5.3min


[CV]  features__tf__topk=500, features__tf_idf__chi__k=40000, score=0.822690992018244, total= 7.8min
[CV]  features__tf__topk=500, features__tf_idf__chi__k=40000, score=0.8134626354820308, total= 7.3min
[CV]  features__tf__topk=500, features__tf_idf__chi__k=40000, score=0.8114285714285714, total= 6.6min


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 64.6min finished


score_1_chi__k:  5000
score_1_topk:  50


([mean: 0.81548, std: 0.00241, params: {'features__tf__topk': 50, 'features__tf_idf__chi__k': 2000},
  mean: 0.81929, std: 0.00328, params: {'features__tf__topk': 50, 'features__tf_idf__chi__k': 5000},
  mean: 0.81739, std: 0.00380, params: {'features__tf__topk': 50, 'features__tf_idf__chi__k': 20000},
  mean: 0.81586, std: 0.00490, params: {'features__tf__topk': 50, 'features__tf_idf__chi__k': 40000},
  mean: 0.81548, std: 0.00241, params: {'features__tf__topk': 100, 'features__tf_idf__chi__k': 2000},
  mean: 0.81929, std: 0.00328, params: {'features__tf__topk': 100, 'features__tf_idf__chi__k': 5000},
  mean: 0.81739, std: 0.00380, params: {'features__tf__topk': 100, 'features__tf_idf__chi__k': 20000},
  mean: 0.81586, std: 0.00490, params: {'features__tf__topk': 100, 'features__tf_idf__chi__k': 40000},
  mean: 0.81548, std: 0.00241, params: {'features__tf__topk': 500, 'features__tf_idf__chi__k': 2000},
  mean: 0.81929, std: 0.00328, params: {'features__tf__topk': 500, 'features__tf_i

In [103]:
pipeline = Pipeline([('features', combined_features), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  objective='multi:softmax', num_class=8))])

param_grid_0 = dict(features__tf_idf__chi__k = [2000, 5000, 20000, 40000 ],# 
                   features__tf__topk=[50,100, 500]) # 
param_grid_0

{'features__tf_idf__chi__k': [2000, 5000, 20000, 40000],
 'features__tf__topk': [50, 100, 500]}

In [ ]:
clf = GridSearchCV(pipeline, param_grid=param_grid_0, 
                   cv = cv, scoring=score_2, n_jobs=-1, verbose=10)
clf.fit(X_train, y_train)
score_2_best_para['chi__k'] = clf.best_params_['features__tf_idf__chi__k']
score_2_best_para['topk'] = clf.best_params_['features__tf__topk']
print('score_2_chi__k: ', score_2_best_para['chi__k'])
print('score_2_topk: ', score_2_best_para['topk'])
clf.grid_scores_, clf.best_params_, clf.best_score_

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] features__tf__topk=50, features__tf_idf__chi__k=2000 ............
[CV] features__tf__topk=50, features__tf_idf__chi__k=2000 ............
[CV] features__tf__topk=50, features__tf_idf__chi__k=2000 ............
[CV] features__tf__topk=50, features__tf_idf__chi__k=5000 ............
[CV]  features__tf__topk=50, features__tf_idf__chi__k=2000, score=0.6637165839517644, total= 3.1min
[CV] features__tf__topk=50, features__tf_idf__chi__k=5000 ............
[CV]  features__tf__topk=50, features__tf_idf__chi__k=2000, score=0.6897705282761166, total= 3.3min
[CV] features__tf__topk=50, features__tf_idf__chi__k=5000 ............
[CV]  features__tf__topk=50, features__tf_idf__chi__k=2000, score=0.6721476122550027, total= 3.4min
[CV] features__tf__topk=50, features__tf_idf__chi__k=20000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=5000, score=0.6966093961388902, total= 4.3min
[CV] features__tf__topk=50, features__tf_i

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  8.9min


[CV]  features__tf__topk=50, features__tf_idf__chi__k=5000, score=0.6604975784755553, total= 4.2min
[CV] features__tf__topk=50, features__tf_idf__chi__k=40000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=20000, score=0.6854724848440027, total= 6.3min
[CV] features__tf__topk=50, features__tf_idf__chi__k=40000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=20000, score=0.6773942476791797, total= 6.6min
[CV] features__tf__topk=50, features__tf_idf__chi__k=40000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=20000, score=0.6702156852743684, total= 6.5min
[CV] features__tf__topk=100, features__tf_idf__chi__k=2000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=40000, score=0.6867400401203034, total= 8.6min


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 18.4min


[CV] features__tf__topk=100, features__tf_idf__chi__k=2000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=40000, score=0.6732463467676054, total= 7.5min
[CV] features__tf__topk=100, features__tf_idf__chi__k=2000 ...........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=2000, score=0.6897705282761166, total= 3.4min
[CV] features__tf__topk=100, features__tf_idf__chi__k=5000 ...........
[CV]  features__tf__topk=50, features__tf_idf__chi__k=40000, score=0.6667103034908572, total= 8.4min
[CV] features__tf__topk=100, features__tf_idf__chi__k=5000 ...........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=2000, score=0.6721476122550027, total= 3.3min
[CV] features__tf__topk=100, features__tf_idf__chi__k=5000 ...........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=2000, score=0.6637165839517644, total= 3.2min
[CV] features__tf__topk=100, features__tf_idf__chi__k=20000 ..........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=5000, score=0.

[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 25.9min


[CV] features__tf__topk=100, features__tf_idf__chi__k=20000 ..........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=5000, score=0.6604975784755553, total= 4.6min
[CV] features__tf__topk=100, features__tf_idf__chi__k=40000 ..........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=20000, score=0.6854724848440027, total= 6.7min
[CV] features__tf__topk=100, features__tf_idf__chi__k=40000 ..........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=20000, score=0.6773942476791797, total= 6.5min
[CV] features__tf__topk=100, features__tf_idf__chi__k=40000 ..........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=20000, score=0.6702156852743684, total= 6.5min
[CV] features__tf__topk=500, features__tf_idf__chi__k=2000 ...........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=40000, score=0.6867400401203034, total= 7.8min
[CV] features__tf__topk=500, features__tf_idf__chi__k=2000 ...........
[CV]  features__tf__topk=500, features__tf_idf__chi__k=2000, scor

[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 39.4min


[CV] features__tf__topk=500, features__tf_idf__chi__k=5000 ...........
[CV]  features__tf__topk=500, features__tf_idf__chi__k=2000, score=0.6721476122550027, total= 3.4min
[CV] features__tf__topk=500, features__tf_idf__chi__k=5000 ...........
[CV]  features__tf__topk=100, features__tf_idf__chi__k=40000, score=0.6667103034908572, total= 7.8min
[CV] features__tf__topk=500, features__tf_idf__chi__k=5000 ...........
[CV]  features__tf__topk=500, features__tf_idf__chi__k=2000, score=0.6637165839517644, total= 3.5min
[CV] features__tf__topk=500, features__tf_idf__chi__k=20000 ..........


## combined_features

In [24]:
combined_features_1 = FeatureUnion([
                                    ('tf_idf', Pipeline([
                                        ('counts', CountVectorizer()),
                                        ('tfidf', TfidfTransformer()),
                                        ('chi', SelectKBest(chi2, k=score_1_best_para['chi__k']))
                                        ])),
                                    ('tf', Statskeywords(topk = score_1_best_para['topk'])),
                                    ('len_stats', StatsFeatures_cor())
                                ])

In [ ]:
combined_features_2 = FeatureUnion([
                                    ('tf_idf', Pipeline([
                                        ('counts', CountVectorizer()),
                                        ('tfidf', TfidfTransformer()),
                                        ('chi', SelectKBest(chi2, k=score_2_best_para['chi__k']))
                                        ])),
                                    ('tf', Statskeywords(topk = score_2_best_para['topk'])),
                                    ('len_stats', StatsFeatures_cor())
                                ])

In [98]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  objective='multi:softmax', num_class=8))])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

0.9998096688237533
accuracy_score:  0.8313359964491789
             precision    recall  f1-score   support

          1       0.81      0.88      0.84       434
          2       0.69      0.46      0.56        97
          3       0.70      0.48      0.57        98
          4       0.85      0.73      0.79       137
          5       0.77      0.71      0.74       177
          6       0.87      0.69      0.77        88
          7       0.84      0.81      0.83       210
          8       0.86      0.93      0.89      1012

avg / total       0.83      0.83      0.83      2253

confusion_matrix: 
[[384   1   0   1   9   2   3  34]
 [ 15  45   0   5   3   2   5  22]
 [  6   3  47   0   5   3   6  28]
 [  8   3   1 100   3   0   0  22]
 [ 22   3   4   0 126   0   9  13]
 [  1   1   3   0   3  61   2  17]
 [  6   4   4   2   4   2 171  17]
 [ 34   5   8   9  10   0   7 939]]


In [ ]:
pipeline = Pipeline([('features', combined_features_2), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  objective='multi:softmax', num_class=8))])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

## n_estimators、learning_rate

In [26]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  objective='multi:softmax', num_class=8))])

param_grid_1 = dict(classifier__n_estimators = range(10,171,30), # 迭代次数/分类器个数
                   classifier__learning_rate=[0.01, 0.1, 0.3]) 
param_grid_1

{'classifier__n_estimators': range(10, 171, 30),
 'classifier__learning_rate': [0.01, 0.1, 0.3]}

In [27]:
clf = GridSearchCV(pipeline, param_grid=param_grid_1, 
                   cv = cv, scoring=score_1, n_jobs=-1)
clf.fit(X_train, y_train)
score_1_best_para['n_estimators'] = clf.best_params_['classifier__n_estimators']
score_1_best_para['learning_rate'] = clf.best_params_['classifier__learning_rate']
print('score_1_n_estimators: ', score_1_best_para['n_estimators'])
print('score_1_learning_rate: ', score_1_best_para['learning_rate'])
clf.grid_scores_, clf.best_params_, clf.best_score_

score_1_n_estimators:  70
score_1_learning_rate:  0.3


([mean: 0.76812, std: 0.00620, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 10},
  mean: 0.78372, std: 0.01000, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 40},
  mean: 0.79056, std: 0.01112, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 70},
  mean: 0.79627, std: 0.01202, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 100},
  mean: 0.79893, std: 0.00989, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 130},
  mean: 0.80179, std: 0.00953, params: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 160},
  mean: 0.79494, std: 0.00763, params: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 10},
  mean: 0.81054, std: 0.00502, params: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 40},
  mean: 0.81701, std: 0.00596, params: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 70},
  mean: 0.81872, std: 0.0060

In [28]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  n_estimators = score_1_best_para['n_estimators'],
                                                  learning_rate = score_1_best_para['learning_rate'],                                                  
                                                  objective='multi:softmax', num_class=8))])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

0.9984782195168347
accuracy_score:  0.8354037267080745
             precision    recall  f1-score   support

          1       0.84      0.88      0.86       456
          2       0.62      0.40      0.48       100
          3       0.70      0.54      0.61        91
          4       0.86      0.71      0.78       132
          5       0.70      0.75      0.72       181
          6       0.85      0.76      0.80        82
          7       0.83      0.81      0.82       207
          8       0.88      0.93      0.90      1005

avg / total       0.83      0.84      0.83      2254

confusion_matrix: 
[[401   8   2   2  14   1   1  27]
 [ 11  40   6   2   9   3   8  21]
 [  4   1  49   1   7   2   5  22]
 [  8   1   0  94   6   0   0  23]
 [ 15   2   3   3 135   1   8  14]
 [  0   1   1   0   3  62   5  10]
 [  6   4   2   0  12   2 167  14]
 [ 32   8   7   7   7   2   7 935]]


In [ ]:
pipeline = Pipeline([('features', combined_features_2), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  objective='multi:softmax', num_class=2))])

param_grid_1 = dict(classifier__n_estimators = range(30,161,20), # 迭代次数/分类器个数
                   classifier__learning_rate=[0.01, 0.1, 0.3]) 
param_grid_1

In [ ]:
clf = GridSearchCV(pipeline, param_grid=param_grid_1, 
                   cv = cv, scoring=score_2, n_jobs=-1)
clf.fit(X_train, y_train)
score_2_best_para['n_estimators'] = clf.best_params_['classifier__n_estimators']
score_2_best_para['learning_rate'] = clf.best_params_['classifier__learning_rate']
print('score_2_n_estimators: ', score_2_best_para['n_estimators'])
print('score_2_learning_rate: ', score_2_best_para['learning_rate'])
clf.grid_scores_, clf.best_params_, clf.best_score_

In [ ]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(max_depth = 7, gamma = 0, 
                                                  n_estimators = score_2_best_para['n_estimators'],
                                                  learning_rate = score_2_best_para['learning_rate'],                                                  
                                                  objective='multi:softmax', num_class=8))])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

## max_depth、min_child_weight

In [29]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(n_estimators = score_1_best_para['n_estimators'],
                                                  learning_rate = score_1_best_para['learning_rate'],
                                                  objective='multi:softmax', num_class=2))])

param_grid_2 = dict(classifier__max_depth=range(3,14,2), # 决策树最大深度
                    classifier__min_child_weight=[4, 5, 6]) 
param_grid_2

{'classifier__max_depth': range(3, 14, 2),
 'classifier__min_child_weight': [4, 5, 6]}

In [30]:
clf = GridSearchCV(pipeline, param_grid=param_grid_2, 
                   cv = cv, scoring=score_1, n_jobs=-1)
clf.fit(X_train, y_train)
score_1_best_para['max_depth'] = clf.best_params_['classifier__max_depth']
score_1_best_para['min_child_weight'] = clf.best_params_['classifier__min_child_weight']
print('score_1_max_depth: ', score_1_best_para['max_depth'])
print('score_1_min_child_weight: ', score_1_best_para['min_child_weight'])
clf.grid_scores_, clf.best_params_, clf.best_score_

score_1_max_depth:  5
score_1_min_child_weight:  4


([mean: 0.81720, std: 0.00696, params: {'classifier__max_depth': 3, 'classifier__min_child_weight': 4},
  mean: 0.81777, std: 0.00553, params: {'classifier__max_depth': 3, 'classifier__min_child_weight': 5},
  mean: 0.81663, std: 0.00853, params: {'classifier__max_depth': 3, 'classifier__min_child_weight': 6},
  mean: 0.82366, std: 0.00396, params: {'classifier__max_depth': 5, 'classifier__min_child_weight': 4},
  mean: 0.81929, std: 0.00435, params: {'classifier__max_depth': 5, 'classifier__min_child_weight': 5},
  mean: 0.81929, std: 0.00475, params: {'classifier__max_depth': 5, 'classifier__min_child_weight': 6},
  mean: 0.81758, std: 0.00233, params: {'classifier__max_depth': 7, 'classifier__min_child_weight': 4},
  mean: 0.82214, std: 0.00560, params: {'classifier__max_depth': 7, 'classifier__min_child_weight': 5},
  mean: 0.82043, std: 0.00555, params: {'classifier__max_depth': 7, 'classifier__min_child_weight': 6},
  mean: 0.81948, std: 0.00286, params: {'classifier__max_depth':

In [33]:
pipeline = Pipeline([('features', combined_features_1), 
                     ('classifier', XGBClassifier(gamma = 0, 
                                                  n_estimators = score_1_best_para['n_estimators'],
                                                  learning_rate = score_1_best_para['learning_rate'],        
                                                  max_depth = score_1_best_para['max_depth'],
                                                  min_child_weight = score_1_best_para['min_child_weight'],                                                   
                                                  objective='multi:softmax', num_class=8))])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

0.9975271067148563
accuracy_score:  0.8345164152617569
             precision    recall  f1-score   support

          1       0.85      0.88      0.86       456
          2       0.56      0.38      0.45       100
          3       0.66      0.54      0.59        91
          4       0.86      0.73      0.79       132
          5       0.73      0.76      0.75       181
          6       0.79      0.73      0.76        82
          7       0.82      0.80      0.81       207
          8       0.88      0.93      0.90      1005

avg / total       0.83      0.83      0.83      2254

confusion_matrix: 
[[400   7   4   2  13   1   2  27]
 [ 15  38   4   2   9   2   7  23]
 [  4   2  49   1   7   2   6  20]
 [  7   3   0  96   4   0   0  22]
 [ 13   5   3   1 138   2   7  12]
 [  0   2   2   0   2  60   6  10]
 [  6   3   4   0  11   4 165  14]
 [ 27   8   8  10   5   5   7 935]]


# 模型保存

In [34]:
from sklearn.externals import joblib
joblib.dump(pipeline, "model/circ_8classifier_0730_1.pkl.z")

['model/circ_8classifier_0730_1.pkl.z']

# 保存本文件

In [56]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_2_train.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)